In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob
import os
import cv2

video_frame_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Resit/sequences/*/')
video_frame_files.sort()
consecutive_frame_pairs_all = []
print(len(video_frame_files))
print(video_frame_files)

62
['/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/animal/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/ants1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/bag/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/ball2/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/ball3/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/basketball/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/birds1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/birds2/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/bolt1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/book/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/bubble/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/butterfly/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/car1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/conduction1/', '/co

In [3]:
for i in range(5):
    color_file = os.path.join(video_frame_files[i], 'color')
    dosyalar = os.listdir(color_file)
    dosyalar.sort()
    jpg_dosyalari = [dosya for dosya in dosyalar if dosya.endswith(".jpg")]
    jpg_dosyalari.sort()

    consecutive_frame_pairs = []
    for i in range(len(jpg_dosyalari) - 1):
    #for i in range(len(jpg_dosyalari)):
         frame_path = os.path.join(color_file, jpg_dosyalari[i])
         second_frame_path = os.path.join(color_file, jpg_dosyalari[i+1])

         frame = cv2.imread(frame_path, cv2.COLOR_BGR2RGB)  # Görüntüyü okuyun
         second_frame = cv2.imread(second_frame_path, cv2.COLOR_BGR2RGB)  # İkinci görüntüyü okuyun

         consecutive_frame_pairs.append((frame, second_frame))
        #consecutive_frame_pairs.append(frame)
    print(len(consecutive_frame_pairs))

    consecutive_frame_pairs_all.append(consecutive_frame_pairs)

99
99
324
310
40


In [4]:
print(len(consecutive_frame_pairs_all[0][0]))

2


In [5]:
import numpy as np
import torch


bounding_box_folder = '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/*/groundtruth.txt'  # Bounding box dosyalarının bulunduğu klasör
bounding_box_datas = []  # Bounding box bilgilerini içeren frame çiftleri listesi

ground_truth_files = glob.glob(bounding_box_folder)
ground_truth_files.sort()
bbox=[]
#print(ground_truth_files)
for file in range(5):

    with open(ground_truth_files[file], 'r') as file:
       bounding_box_lines = file.readlines()
       new_box=[]
       for i in range(len(bounding_box_lines)-1):
          bbox_values_1 = bounding_box_lines[i].strip().split(',')
          bbox_values_2 = bounding_box_lines[i+1].strip().split(',')
          # Bounding box değerlerini float'a dönüştürün
          bbox_values1 = [float(value) for value in bbox_values_1]
          bbox_values2 = [float(value) for value in bbox_values_2]
          bbox_tensor_2 = torch.tensor(bbox_values2, dtype=torch.float32)

          bbox.append(bbox_tensor_2)
          #print(resized_bounding_box)
          new_box.append((bbox_values1, bbox_values2))
       bounding_box_datas.append(new_box)
       #print(len(new_box))

       #print(len(bounding_box_lines))


#print(bounding_box_datas[0])
print(len(bounding_box_datas[0]))



99


In [6]:
print(bounding_box_datas[0][0][1])
print(len(bbox))
print(type(bbox[0]))



[553.0, 242.0, 24.0, 40.0]
872
<class 'torch.Tensor'>


In [7]:
def crop_frame(frame, bbox):
    #bbox=np.array(list(map(float, bbox.split(','))))
     # get the bbox values
    x, y, width, height = bbox
    #x, y, width, height = bbox
    frame_height, frame_width, _ = frame.shape

    if height > frame_height:
        height = frame_height

    if width > frame_width:
        width = frame_width

    # Orta noktayı hesaplayalım
    center_x = x + (width / 2)
    center_y = y + (height / 2)

    # Büyütülmüş bbox boyutlarını hesaplayalım
    enlarged_width = width * 2
    enlarged_height = height * 2

    # Kırpılmış çerçeveyi hesaplayalım
    cropped_frame = frame[int(center_y - (enlarged_height / 2)):int(center_y + (enlarged_height / 2)),
                          int(center_x - (enlarged_width / 2)):int(center_x + (enlarged_width / 2))]

    return cropped_frame

In [8]:
##CROP IMAGE
cropped_images1=[]
cropped_images2=[]
cropped_images3=[]
for i in range(len(consecutive_frame_pairs_all)):
  for j in range(len(consecutive_frame_pairs_all[i])):
    frame1_cropped=crop_frame(consecutive_frame_pairs_all[i][j][0], bounding_box_datas[i][j][0])
    frame2_cropped_FALSE=crop_frame(consecutive_frame_pairs_all[i][j][1], bounding_box_datas[i][j][0])
    frame2_cropped_TRUE=crop_frame(consecutive_frame_pairs_all[i][j][1], bounding_box_datas[i][j][1])
    cropped_images1.append(frame1_cropped)
    cropped_images2.append(frame2_cropped_FALSE)
    cropped_images3.append(frame2_cropped_TRUE)

print(len(cropped_images1))

872


In [37]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, array1, array2, array3):
        self.array1 = array1
        self.array2 = array2
        self.array3 = array3

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((225, 225)),
            transforms.ToTensor()
        ])

    def __getitem__(self, index):
        input1 = self.transform(self.array1[index])
        input2 = self.transform(self.array2[index])
        target = self.transform(self.array3[index])

        return input1, input2, target

    def __len__(self):
        return len(self.array1)


In [15]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Resize
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, cropped_images1, cropped_images2, bbox):
        self.cropped_images1 = cropped_images1
        self.cropped_images2 = cropped_images2
        self.bbox = bbox
        self.resize_transform = Resize((225, 225))
        self.to_tensor = ToTensor()

    def __getitem__(self, index):

        frame1 = self.cropped_images1[index]
        frame2 = self.cropped_images2[index]
        bbox = self.bbox[index]

        # Görüntüleri PIL.Image formatına dönüştürün
        pil_frame1 = Image.fromarray(frame1)
        pil_frame2 = Image.fromarray(frame2)

        # Görüntüleri 225x225 boyutuna yeniden boyutlandırın
        resized_frame1 = self.resize_transform(pil_frame1)
        resized_frame2 = self.resize_transform(pil_frame2)

        # Gerekli işlemleri gerçekleştirerek birleştirilmiş girdi ve hedef çıktıyı döndürün
        combined_frame = torch.cat([self.to_tensor(resized_frame1), self.to_tensor(resized_frame2)], dim=1)
        combined_frame = combined_frame.unsqueeze(0)

        target = torch.tensor(bbox, dtype=torch.float32)

        return combined_frame, target

    def __len__(self):
        return len(self.cropped_images1)


In [16]:

#frame1_dataset = CustomDataset(cropped_images1)
#frame2_dataset = CustomDataset(cropped_images2)
#frame3_dataset = CustomDataset(cropped_images3)
#print(type(frame1_dataset[0]))
#train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#dataset = CustomDataset(cropped_images1, cropped_images2, cropped_images3)
dataset = CustomDataset(cropped_images1, cropped_images2, bbox)

batch_size = 16  # Örnekleme sayısı
shuffle = True  # Veriyi karıştır
data_loader = torch.utils.data.DataLoader( dataset, batch_size=batch_size, shuffle=shuffle)


In [18]:
import torch
from torchvision.models import resnet34
from torch.utils.data import DataLoader

# Önceden tanımlanmış ResNet-34 modelini yükleyin
model = resnet34(pretrained=True)

# Son konvolüsyonel katmana kadar olan kısmı alın
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

# Tam bağlantılı katmanları tanımlayın
hidden_size = 512
output_size = 4  # (x, y, width, height)
fc1 = torch.nn.Linear(hidden_size * 2, hidden_size)
fc2 = torch.nn.Linear(hidden_size, output_size)

# Loss fonksiyonu ve optimize edici
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



# Eğitim döngüsü
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0

    for frames, targets in data_loader:
        # Verileri uygun cihaza taşıyın
        frame1 = frames[0].to(device)
        frame2 = frames[1].to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        # Özellik çıkarımı
        features1 = feature_extractor(frame1)
        features2 = feature_extractor(frame2)

        # Özellikleri birleştirin
        combined_features = torch.cat((features1.flatten(start_dim=1), features2.flatten(start_dim=1)), dim=1)

        # Tahmin yapın
        hidden = torch.relu(fc1(combined_features))
        predictions = fc2(hidden)

        # Beklenen bounding box koordinatları
        ground_truth_box = targets

        # Loss hesaplayın ve geriye doğru yayılım yapın
        loss = criterion(predictions, ground_truth_box)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(data_loader)
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-15-e72f8427b54e>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(bbox, dtype=torch.float32)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a tar

ValueError: ignored

In [42]:
import torch
from torchvision.models import resnet34

# Install the pre-trained ResNet-34 model
model = resnet34(pretrained=True)

# Train Model
criterion = torch.nn.MSELoss()  # Kayıp fonksiyonu olarak MSE kullanıyoruz
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # SGD optimizerini kullanıyoruz

epoch=10 #train for 10 epochs
for epoch in range(epoch):
    running_loss = 0.0

    for frame1, frame2, frame3 in data_loader:
        optimizer.zero_grad()

        # Clipping and feature extraction for the first frame
        clipped_frame1 = torch.clamp(frame1, 0, 1)  # Görüntüyü [0, 1] aralığında kırpmak
        features1 = model(clipped_frame1)  # Feature extraction

        # Clipping and feature extraction for the second frame
        clipped_frame2 = torch.clamp(frame2, 0, 1)  # Görüntüyü [0, 1] aralığında kırpmak
        features2 = model(clipped_frame2)  # Feature extraction

        # Combining feature vectors
        combined_features = torch.cat((features1, features2), dim=1)

        # Resize predictions to match the size of target frame
        resized_frame3 = torch.nn.functional.interpolate(frame3, size=(frame2.size(2), frame2.size(3)), mode='bilinear')

        # Loss calculation and back propagation
        loss = criterion(combined_features, resized_frame3)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(data_loader)
    print(f"Epoch {epoch+1} Loss: {epoch_loss}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16, 3, 225, 225])) that is different to the input size (torch.Size([16, 2000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [46]:
import torch
from torchvision.models import resnet34

# Install the pre-trained ResNet-34 model
model = resnet34(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

# Train Model
criterion = torch.nn.MSELoss()  # Kayıp fonksiyonu olarak MSE kullanıyoruz
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # SGD optimizerini kullanıyoruz

epoch=10 #train for 10 epochs
for epoch in range(epoch):
    running_loss = 0.0

    for frame1, frame2, frame3 in data_loader:
        optimizer.zero_grad()

        # Clipping and feature extraction for the first frame

        features1 = model(frame1)  # Feature extraction


        features2 = model(frame2)  # Feature extraction

        # Combining feature vectors
        combined_features = torch.cat((features1, features2), dim=1)
        print(combined_features.shape)
        print(frame3.shape)
        #num_features = combined_features.size(1)
        #fc = torch.nn.Linear(num_features, num_classes)
        # Predictions
        predictions = combined_features
        print(predictions)


        # lass calculation and back propagation
        loss = criterion(combined_features, frame3)
        loss.backward()


        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(data_loader)
    print(f"Epoch {epoch+1} Loss: {epoch_loss}")


torch.Size([16, 2000])
torch.Size([16, 3, 225, 225])
tensor([[ 2.7168,  2.5712, -0.4412,  ..., -2.6682,  2.7235,  2.0518],
        [-1.0332, -1.7125, -0.6044,  ..., -3.0836,  0.3117,  0.4884],
        [ 0.7137,  0.9680,  0.3239,  ...,  2.3345,  0.3345, -0.2982],
        ...,
        [-1.5857, -0.5992, -0.2391,  ..., -2.6184,  3.5763,  3.8493],
        [-1.0646, -2.0272, -0.3754,  ...,  0.2550,  1.1468,  1.7144],
        [-1.8155, -3.1302,  0.4117,  ...,  1.1284,  0.1644,  0.5099]],
       grad_fn=<CatBackward0>)


RuntimeError: ignored

In [ ]:
cropped_frame1=[]
for i in range(10):
  cropped_frames=[]
  for j in range(len(frame2_path_all[i])):
    ground_truth=np.array(list(map(float, bounding_box1_all[i][j].split(','))))
    enlarged_bbox = ground_truth * 2  # Bounding box'u 2 kat büyütün
    bbox_center = (ground_truth[0] + ground_truth[2] / 2, ground_truth[1] + ground_truth[3] / 2)  # Bounding box'un merkezini hesaplayın
    bbox_size = (enlarged_bbox[2], enlarged_bbox[3])  # Bounding box'un boyutunu alın
    x1 = int(bbox_center[0] - bbox_size[0] / 2)  # Bounding box'un sol üst köşesinin x koordinatını hesaplayın
    y1 = int(bbox_center[1] - bbox_size[1] / 2)  # Bounding box'un sol üst köşesinin y koordinatını hesaplayın
    x2 = int(bbox_center[0] + bbox_size[0] / 2)  # Bounding box'un sağ alt köşesinin x koordinatını hesaplayın
    y2 = int(bbox_center[1] + bbox_size[1] / 2)  # Bounding box'un sağ alt köşesinin y koordinatını hesaplayın

    cropped_frame = frame[y1:y2, x1:x2]
    cropped_frames.append(cropped_frame)
  print(len(cropped_frames))
  cropped_frame1.append(cropped_frames)
print(len(cropped_frame1))



99
99
324
310
40
170
724
338
79
349
10


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim

# 1. Eğitim veri yükleyicisi oluşturma
# TODO: Eğitim veri yükleyicisini hazırlayın

# 2. Frame kırpması
def crop_frame(frame, bbox):
    x1 = int(bbox[0] - bbox[2] / 2)
    y1 = int(bbox[1] - bbox[3] / 2)
    x2 = int(bbox[0] + bbox[2] / 2)
    y2 = int(bbox[1] + bbox[3] / 2)
    return frame[y1:y2, x1:x2]

# 3. Pre-trained ResNet-34 modelinin yüklenmesi
model = models.resnet34(pretrained=True)
model.eval()

# 4. Özellik çıkarma
def extract_features(image, model):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        features = model(input_batch)
    return features

# 5. Özellik birleştirme
class FeatureConcatenation(nn.Module):
    def __init__(self):
        super(FeatureConcatenation, self).__init__()

    def forward(self, features1, features2):
        return torch.cat((features1, features2), dim=1)

# 6. Tam bağlantılı katmanlar ve kayıp fonksiyonu
class BoundingBoxPrediction(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BoundingBoxPrediction, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 4)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

# Hyperparametreler
input_size = 2 * model.fc.in_features
hidden_size = 512
learning_rate = 0.001
num_epochs = 10

# Model oluşturma
concatenation = FeatureConcatenation()
bounding_box_prediction = BoundingBoxPrediction(input_size, hidden_size)

# Optimizer ve kayıp fonksiyonu
optimizer = optim.Adam(list(concatenation.parameters()) + list(bounding_box_prediction.parameters()), lr=learning_rate)
criterion = nn.MSELoss()

# Eğitim döngüsü
for epoch in range(num_epochs):
    for frames in dataloader:  # 1. adımda oluşturulan eğitim veri yükleyicisini kullanarak frame çiftlerini alın
        t0, t1 = frames  # Örnek olarak sadece iki frame kullanıldı

        # 2. Frame kırpması
        cropped_frame1 = crop_frame(t0, padded_ground_truth_bbox)  # İlk frame'in kırpılması
        cropped_frame2 = crop_frame(t1, search_region_bbox)  # İkinci frame'in kırpılması

        # 3. Özellik çıkarma
        features1 = extract_features(cropped_frame1, model)
        features2 = extract_features(cropped_frame2, model)

        # 4. Özellik birleştirme
        concatenated_features = concatenation(features1, features2)

        # 5. Tahmin ve kayıp hesaplama
        predicted_bbox = bounding_box_prediction(concatenated_features)
        loss = criterion(predicted_bbox, ground_truth_bbox)

        # Geriye yayılım ve optimizasyon
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

# Eğitimi tamamladıktan sonra, modeliniz eğitilmiş olacak ve tahminler yapabilirsiniz.


In [ ]:
import cv2

frame1_cropped=[]
print(len(bounding_box1_all))
for frame1_path, bounding_box1 in zip(frame1_path_all, bounding_box1_all):

  #print(frame1_path)
  frame1_img=[]
  for frame1_path_ , bounding_box1_ in zip(frame1_path, bounding_box1):
     frame1= cv2.imread(frame1_path_)
     #print(frame1)
     #print(bounding_box1_)
     x1, y1, w1, h1 = map(float, bounding_box1_.split(','))
     frame_height, frame_width, _ = frame1.shape  # Kare boyutunu alın
     center_x = x1 + w1 / 2  # Bounding box'un merkezinin x koordinatı
     center_y = y1 + h1 / 2  # Bounding box'un merkezinin y koordinatı
     enlarged_width = int(w1 * 2)  # Genişlik 2 kat büyütülecek
     enlarged_height = int(h1 * 2)  # Yükseklik 2 kat büyütülecek
     x_crop = int(center_x - enlarged_width / 2)  # Kırpma için sol üst köşenin x koordinatı
     y_crop = int(center_y - enlarged_height / 2)  # Kırpma için sol üst köşenin y koordinatı

     # Kırpma işlemini gerçekleştirin
     cropped_frame = frame1[y_crop:y_crop + enlarged_height, x_crop:x_crop + enlarged_width]
     frame1_img.append(cropped_frame)
  frame1_cropped.append(frame1_img)
  print(len(frame1_img))







# İlk frame'i yükleyin
#frame1 = cv2.imread(frame1_path)

# Ground-truth sınırlayıcı kutudan koordinatları alın
#x1, y1, w1, h1 = bounding_box1

# Sınırlayıcı kutuyu 2 kat büyütün
#x1 -= int(w1 / 2)
#y1 -= int(h1 / 2)
#w1 *= 2
#h1 *= 2

# Büyütülmüş sınırlayıcı kutuyla ilk frame'i kırpın
#cropped_frame1 = frame1[y1:y1+h1, x1:x1+w1]


10


TypeError: ignored

In [ ]:
import cv2
def read_img( path):
  new_arr_img=[]
  for img in path:
    img_arr=cv2.imread(img)
    new_arr_img.append(img_arr)

  return new_arr_img



In [ ]:
frame1_img_all=[]

In [ ]:
import cv2

for i in range(0,15):
  print(frame1_path_all[i])
  frame1_img_all.append(read_img(frame1_path))


['/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000001.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000002.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000003.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000004.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000005.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000006.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000007.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000008.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000009.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000010.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000011.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/

In [ ]:
import cv2

for frame1_path in range(len(frame1_path_all)/4,len(frame1_path_all)/2):
  print(frame1_path[0])
  frame1_img_all.append(read_img(frame1_path))

In [ ]:
import cv2

for frame1_path in range(len(frame1_path_all)/2,3*(len(frame1_path_all)/4)):
  print(frame1_path[0])
  frame1_img_all.append(read_img(frame1_path))

In [ ]:
import cv2

for frame1_path in range(3*(len(frame1_path_all)/4),len(frame1_path_all)):
  print(frame1_path[0])
  frame1_img_all.append(read_img(frame1_path))

In [ ]:
# İkinci frame'i yükleyin
frame2 = cv2.imread(frame2_path)

# Önceki frame'in sınırlayıcı kutusundan koordinatları alın
x2, y2, w2, h2 = bounding_box2

# Sınırlayıcı kutuyu 2 kat büyütün
x2 -= int(w2 / 2)
y2 -= int(h2 / 2)
w2 *= 2
h2 *= 2

# Büyütülmüş sınırlayıcı kutuyla ikinci frame'i kırpın
cropped_frame2 = frame2[y2:y2+h2, x2:x2+w2]
